In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
devtools::load_all("/Users/kimja/university/PhD/projects/pathway_regression/tool/pareg")

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

from tqdm.auto import trange

In [ ]:
sns.set_context('talk')

# Basic test

## Generate data

In [ ]:
N = 10
df_genes = pd.DataFrame({
    'gene': [f'g{i:02}' for i in range(N)],
    'pvalue': np.random.beta(0.1, 1, N)
})
df_genes.head()

In [ ]:
df_terms = pd.DataFrame({
    'name': ['foo'] * 10,
    'gene': [f'g{i:02}' for i in range(5, 15)],
})
df_terms.head()

## Run model

In [ ]:
%%R -i df_genes,df_terms -o df_enrich

df_enrich <- pareg(df_genes, df_terms)

In [ ]:
df_enrich.head()

# Investigate enrichment patterns

In [ ]:
df_terms = pd.DataFrame({
    'name': ['foo'] * 100,
    'gene': [f'g{i:02}' for i in range(100, 200)],
})

df_terms.head()

In [ ]:
window_size = 10
genes = [f'g{i:02}' for i in range(90, 110)]
repetition_num = 10

tmp_list = []
%Rpush df_terms
for idx in trange(10):
    for rep in trange(repetition_num, leave=False):
        # generate genes
        p_values = np.concatenate([
            np.random.beta(1, 1, idx), # uniform
            np.random.beta(0.1, 1, window_size), # "significant" p-values
            np.random.beta(1, 1, 20-idx-window_size) # uniform
        ])
        tmp_genes = pd.DataFrame({
            'gene': genes,
            'pvalue': p_values
        })

        # compute enrichment
        %Rpush tmp_genes
        tmp = %R pareg(tmp_genes, df_terms)

        tmp['idx'] = idx
        tmp['repetition_idx'] = rep
        tmp_list.append(tmp)
df_enrich = pd.concat(tmp_list, ignore_index=True)

In [ ]:
df_enrich.head()

$g_i^{(s)} \in G^{(s)} \rightarrow y_i^{(s)} \sim \mathcal{B}(0.1, 1)$

$g_i^{(ns)} \in G^{(ns)} \rightarrow y_i^{(ns)} \sim \mathcal{B}(1, 1)$

Pathway (set of genes) $P$

In [ ]:
plt.figure(figsize=(8,6))

sns.lineplot(
    x='idx', y='enrichment', data=df_enrich, 
    marker='o')

plt.xlabel('Data composition')
plt.xticks(
    [
        df_enrich['idx'].min(), 
        df_enrich['idx'].max()
    ],
    [
        r'$G^{(s)} \cap P = \emptyset \wedge G^{(ns)} \subseteq P$', 
        r'$G^{(ns)} \cap P = \emptyset \wedge G^{(s)} \subseteq P$'
    ]
)

plt.ylabel('Enrichment score')

plt.savefig('enrichment_example.pdf')